In [1]:
%load_ext autoreload
%autoreload 2
from new_aeb_gplvm import *
from utils.data_generator import DataGenerator
from utils.myutils import Utils
from tqdm import trange
from sklearn.preprocessing import StandardScaler, MinMaxScaler
import matplotlib.pyplot as plt
import random
from tqdm import tqdm_notebook
random.seed(42)
import json
import os
datagenerator = DataGenerator()
utils = Utils()

2023-11-29 01:35:47.406805: I tensorflow/core/platform/cpu_feature_guard.cc:193] This TensorFlow binary is optimized with oneAPI Deep Neural Network Library (oneDNN) to use the following CPU instructions in performance-critical operations:  SSE4.1 SSE4.2 AVX AVX2 AVX_VNNI FMA
To enable them in other operations, rebuild TensorFlow with the appropriate compiler flags.
2023-11-29 01:35:47.455030: I tensorflow/core/util/port.cc:104] oneDNN custom operations are on. You may see slightly different numerical results due to floating-point round-off errors from different computation orders. To turn them off, set the environment variable `TF_ENABLE_ONEDNN_OPTS=0`.


In [2]:
EXPERIMENTS_FILE = "experiments/normal/004_latent_inducing_experiments.json"
with open(EXPERIMENTS_FILE) as file:
    experiments = json.load(file)

In [3]:
def create_dataframe(dataset):
    datagenerator.dataset = dataset
    data = datagenerator.generator(
        la=1.00,
        realistic_synthetic_mode=None,
        stdscale=True,
        minmax=False,
        noise_type=None,
    )
    Y_train = torch.tensor(data["X_train"], dtype=torch.float32)
    Y_test = torch.tensor(data["X_test"], dtype=torch.float32)
    lb_train = torch.tensor(data["y_train"], dtype=torch.float32)
    lb_test = torch.tensor(data["y_test"], dtype=torch.float32)
    #idx_n = np.where(lb_train == 0)[0]
    #Y_train = Y_train[idx_n]
    #lb_train = lb_train[idx_n]
    return Y_train, Y_test, lb_train, lb_test

In [4]:
experiments[0]

{'dataset': '01_ALOI',
 'kernel': 'rbf',
 'layers': '5,5',
 'latent_dim': 2,
 'lr': 0.01,
 'batch_size': 128,
 'n_inducing': 50,
 'n_epochs': 1000}

In [ ]:
for experiment in tqdm_notebook(experiments):
    if "auc_roc" not in experiment.keys():
        Y_train, Y_test, lb_train, lb_test = create_dataframe(experiment["dataset"])
        n_train = len(Y_train)
        data_dim = Y_train.shape[1]
        latent_dim = experiment["latent_dim"]
        nn_layers = tuple(map(int, experiment["layers"].split(",")))
        kernel = experiment["kernel"]
        n_inducing = experiment["n_inducing"]

        n_epochs = 1000
        lr = 0.01
        batch_size = 128

        try:
            gplvm = AD_GPLVM(
                latent_dim, n_inducing, n_epochs, nn_layers, lr, batch_size
            )
            gplvm.fit(Y_train)
            score = gplvm.predict_score(Y_test)
            metrics = utils.metric(y_true=lb_test, y_score=score)

            experiment["auc_roc"] = metrics["aucroc"]
            experiment["auc_pr"] = metrics["aucpr"]
        except:
            experiment["auc_roc"] = 0.00
            experiment["auc_pr"] = 0.00

        experiment["lr"] = lr
        experiment["batch_size"] = batch_size
        experiment["n_inducing"] = n_inducing
        experiment["n_epochs"] = n_epochs
        # experiment["train_noise"] =  float(lb_train.sum()/len(Y_train))

        with open("experiments/normal/004_latent_inducing_results.json", "w") as file:
            json.dump(experiments, file)